# Подгрузка пакетов и данных

In [1]:
cd ..

/Users/n.barsukov/WebstormProjects/ml-bug-detector


In [2]:
import json

from gensim.models.word2vec import Word2Vec

from shared.consts import FUNCTIONS_JSON_PROPERTIES_NAMES, PATH_TO_FUNCTIONS_STORAGE_JSON, PATH_TO_WORD2VEC_MODEL

Загрузим уже построенную модель Word2Vec

In [4]:
word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

Загрузим JSON, в котором мы сложили названия функций (и названия их арггументов)

In [5]:
functions_storage_dictionary = json.load(open(PATH_TO_FUNCTIONS_STORAGE_JSON))

# Отделение объявления функций от их использования

In [11]:
def find_all_functions_creations(func_storage_dic):
    func_creation_dic = {}
    
    for fileName in func_storage_dic.keys():
        for function in func_storage_dic[fileName]:
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionDeclaration' or
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionExpression'
               ):
                if function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] in func_creation_dic:
                    pass
                    # print(function[PROPERTY_WITH_FUNC_NAME], 'уже есть в словаре')
                func_creation_dic[function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]] = function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]
    
    return func_creation_dic

In [12]:
func_creation_dic = find_all_functions_creations(functions_storage_dictionary)

А теперь находим все ВЫЗОВЫ функций и смотрим, насколько их аргументы отличны объявления функции

In [13]:
def find_all_func_executions(func_storage_dic, func_creation_dic):
    func_exec_storage = []
    
    for fileName in func_storage_dic.keys():
        for function in func_storage_dic[fileName]:
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'CallExpression' and # должна быть вызовом функции
                len(function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) > 0 and # должна иметь хоть один аргумент
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] in func_creation_dic # должна быть в нашем словаре
            ):
                func_exec_storage.append(function)
    
    return func_exec_storage

In [14]:
func_exec_storage = find_all_func_executions(functions_storage_dictionary, func_creation_dic)

In [15]:
print('Размер словаря с объявлениями функций: ', len(func_creation_dic))
print('Размер массива с вызовами функций: ', len(func_exec_storage))

Размер словаря с объявлениями функций:  16927
Размер массива с вызовами функций:  5600


Словарь с объявлениями функций - словарь, в котором ключ - название функции, а значение – массив с названиями переданных аргументов.

Пример: функция с названием **confirmDelete**

In [16]:
func_creation_dic['confirmDelete']

['url', 'subject', 'reloadAfter']

Массив с вызовами функций состоит из словарей следующего формата

In [17]:
func_exec_storage[0]

{'type': 'CallExpression',
 'argumentsNames': ['user_id'],
 'functionName': 'set'}

# Алгоритм проверки схожести аргументов

In [18]:
def checkIfWordInWord2VecDict(word, word2VecVocabulary = word2VecModel.wv):
    return word in word2VecVocabulary

def get_args_similarity(exec_func_name, exec_func_args_names , func_dict, word2VecModel):
    """Определяет схожесть названий аргументов в вызове функции с названиями аргументов в объявлении функции"""
    args_similarity = []
    
    for argIndex in range(len(exec_func_args_names)):
        if argIndex > len(func_dict[exec_func_name]) - 1:
            args_similarity.append(None)
            continue
        
        exec_arg = exec_func_args_names[argIndex]
        create_arg = func_dict[exec_func_name][argIndex]
        
        if (checkIfWordInWord2VecDict(exec_arg) and checkIfWordInWord2VecDict(create_arg)):
            args_similarity.append(word2VecModel.wv.similarity(exec_arg, create_arg))
        else:
            args_similarity.append(None)
        
    return args_similarity

Посмотрим, как это работает на примере

In [19]:
example_func_exec = list(filter(lambda func: func[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] == 'updateLine', func_exec_storage))
# print(example_func_exec)
example_func_exec = example_func_exec[0]

func_name = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
args_names = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]

print('Вызов функции:', func_name, 'c аргументами', args_names)
print('Функция же была объявлена с словаре с аргументами:', func_creation_dic[func_name])

get_args_similarity(func_name, args_names, func_creation_dic, word2VecModel)

Вызов функции: updateLine c аргументами ['lastLine', 'lastSpans', 'estimateHeight']
Функция же была объявлена с словаре с аргументами: ['line', 'text', 'markedSpans', 'estimateHeight']


[0.48107764, 0.061740745, 0.3845044]

# Реализация на всем массиве вызовов функции

Идея максимально проста. Проходим по массиву вызовов функции.

- По названию функции определяем в словаре с объявлениями функций, с какими названиями аргументов была объявлена эта функция
- Для каждой пары аргументов (название аргумента в момент вызовы функции и название аргумента в момент объявление функции) считаем близость их названий в векторной форме с помощью уже построенной модели word2Vec (это будет значение 0 до 1)
- Если эта схожесть меньше заданного порога (threshold), то выбрасываем предупреждение пользователю

### Threshold

In [21]:
MIN_SIMILARITY_THRESHOLD = 0.1

Порог не должен быть большой, потому что модели важно достичь высокого precision, а не recall.

Если наш алгоритм будет часто беспокоить пользователя по пустякам, предупреждая об ошибках там, где их нет (false positive), то нашим алгоритмом просто никто не будет пользоваться. Нужно беспокоить программиста только прям в самых вероятных случаях наличия ошибки. Ведь программисты люди ленивые и занятые... 

In [22]:
for func_exec in func_exec_storage:
    func_name = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
    args_names = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]
    
    args_similarities = get_args_similarity(func_name, args_names, func_creation_dic, word2VecModel)
    is_below_threshold_args_similarities = list(map(lambda x: x and x < MIN_SIMILARITY_THRESHOLD, args_similarities))
    
    if (any(is_below_threshold_args_similarities)):
        print(func_name, args_names, '|', func_creation_dic[func_name])

extend ['JSON'] | ['subclass', 'superclass']
define ['BitfieldT'] | ['factory']
define ['PointerT'] | ['factory']
define ['PointerT', 'PointeeType'] | ['factory']
each ['values'] | ['o', 'f', 'old_logic']
api ['reified'] | ['url', 'method', 'data']
each ['cssSelector'] | ['o', 'f', 'old_logic']
inherits ['AppDispatcher'] | ['child', 'parent']
push ['$default_selector_img_bounds_margin$$inline_746_parent$$inline_753$$'] | ['m']
$goog$inherits$$ ['$annotorious$mediatypes$image$ImageModule$$', '$annotorious$mediatypes$Module$$'] | ['$childCtor$$', '$parentCtor$$']
$goog$inherits$$ ['$annotorious$mediatypes$openlayers$OpenLayersModule$$', '$annotorious$mediatypes$Module$$'] | ['$childCtor$$', '$parentCtor$$']
inherits ['Visitor'] | ['child', 'parent']
extend ['AliasInterfaceError', 'Error'] | ['subclass', 'superclass']
extend ['CustomError', 'Error'] | ['subclass', 'superclass']
implement ['DecimalFormatter', 'Formatter'] | ['proto', 'value']
implement ['LowerCaseFormatter', 'Formatter'] |

remove ['a'] | ['credentials', 'account', 'callback']
remove ['a'] | ['credentials', 'account', 'callback']
extend ['a'] | ['subclass', 'superclass']
extend ['b'] | ['subclass', 'superclass']
extend ['a'] | ['subclass', 'superclass']
extend ['b'] | ['subclass', 'superclass']
extend ['c'] | ['subclass', 'superclass']
extend ['d', 'arrayLikeObject'] | ['subclass', 'superclass']
extend ['e', 'emptyArrayLikeObject'] | ['subclass', 'superclass']
extend ['f', 'length3ArrayLikeObject', 'length3ArrayLikeObject'] | ['subclass', 'superclass']
extend ['result', 'bigArray'] | ['subclass', 'superclass']
extend ['a'] | ['subclass', 'superclass']
extend ['a', 'd'] | ['subclass', 'superclass']
sort ['c'] | ['modules']
sort ['c'] | ['modules']
sort ['e'] | ['modules']
sort ['e'] | ['modules']
sort ['h'] | ['modules']
sort ['h'] | ['modules']
moveItem ['arguments'] | ['oldIndex', 'newIndex']
inherits ['SubClass', 'BaseClass'] | ['child', 'parent']
inherits ['C', 'B'] | ['child', 'parent']
inherits ['C',

_free ['$2'] | ['$mem']
_free ['$3'] | ['$mem']
_free_html_label ['$1'] | ['$lp', '$root']
_free_html_data ['$2'] | ['$dp']
_free ['$3'] | ['$mem']
__printXDot ['$x', '$1'] | ['$x', '$print', '$info']
_gvrender_polygon ['$job', '$1', '$filled'] | ['$job', '$af', '$n', '$filled']
_free ['$1'] | ['$mem']
_free_html_font ['$5'] | ['$fp']
_agxbinit ['$str', '$11'] | ['$xb', '$hint', '$init']
_agxbfree ['$str'] | ['$xb']
_free_html_data ['$5'] | ['$dp']
__max ['$3', '$4', '$5'] | ['$yval', '$v0', '$v1']
__min ['$13', '$4', '$5'] | ['$yval', '$v0', '$v1']
_dup_proto ['$g', '$2'] | ['$g', '$proto']
_agFREEdict ['undef', '$5'] | ['$g', '$dict']
_agFREEdict ['undef', '$9'] | ['$g', '$dict']
_agFREEdict ['undef', '$13'] | ['$g', '$dict']
_aglexinit ['$2'] | ['$fp', '$mygets']
_copydict ['$8', '$5'] | ['$from', '$to']
_copydict ['$15', '$11'] | ['$from', '$to']
_copydict ['$36', '$28'] | ['$from', '$to']
___cxa_throw ['$1', '__ZTISt9bad_alloc'] | ['ptr', 'type', 'destructor']
_free ['start'] | ['

search ['elt'] | ['keywords', 'callback']
inherits ['ReadStream', 'Transform'] | ['child', 'parent']
inherits ['WriteStream', 'Transform'] | ['child', 'parent']
extend ['VK'] | ['subclass', 'superclass']
list ['dbOptions'] | ['args']
extend ['ev', 'o'] | ['subclass', 'superclass']
each ['newUserDoc'] | ['o', 'f', 'old_logic']
debug ['prop', 'ret'] | ['message', 'showHidden']
debug ['prop', 'style'] | ['message', 'showHidden']
each ['values'] | ['o', 'f', 'old_logic']
bind ['document'] | ['prop1', 'prop2']
fn ['router', 'byId'] | ['type', 'item', 'cnt', 'H']
removeLayer ['startMarker'] | ['name', 'group']
inherits ['daemon', 'EventEmitter'] | ['child', 'parent']
inherits ['daemon', 'EventEmitter'] | ['child', 'parent']
inherits ['N1qlStringQuery', 'N1qlQuery'] | ['child', 'parent']
play ['onError'] | ['array', 'x', 'y', 'move']
add ['plural'] | ['manager', 'name', 'password']
push ['header'] | ['m']
push ['removeData'] | ['m']
push ['removeData'] | ['m']
add ['values', 'index', 'n'] | [

__ZN15b2RevoluteJoint9SetLimitsEff ['$4', '$5', '$6'] | ['$this', '$lower', '$upper']
__ZN7b2Joint11SetUserDataEPv ['$4', '$5'] | ['$this', '$data']
__ZN14b2PolygonShape8SetAsBoxEffRK6b2Vec2f ['$6', '$7', '$8', '$9', '$10'] | ['$this', '$hx', '$hy', '$center', '$angle']
__ZN9b2Contact10SetEnabledEb ['$4', '$6'] | ['$this', '$flag']
__ZNK9b2Contact16GetWorldManifoldEP15b2WorldManifold ['$3', '$4'] | ['$this', '$worldManifold']
__ZN15b2WorldManifold10InitializeEPK10b2ManifoldRK11b2TransformfS5_f ['$16', '$17', '$19', '$22', '$24', '$27'] | ['$this', '$manifold', '$xfA', '$radiusA', '$xfB', '$radiusB']
__ZN6b2Body9SetActiveEb ['$4', '$6'] | ['$this', '$flag']
__ZN6b2Body15SetGravityScaleEf ['$3', '$4'] | ['$this', '$scale']
__ZN6b2Body11SetUserDataEPv ['$3', '$4'] | ['$this', '$data']
__ZN6b2Body10ApplyForceERK6b2Vec2S2_ ['$4', '$5', '$6'] | ['$this', '$force', '$point']
__ZN6b2Body18SetSleepingAllowedEb ['$4', '$6'] | ['$this', '$flag']
__ZN6b2Body12SetTransformERK6b2Vec2f ['$4', '$5', '

setupMethods ['web3', 'web3Methods'] | ['obj', 'methods']
setupProperties ['web3', 'web3Properties'] | ['obj', 'properties']
inherits ['EtherSheetService'] | ['child', 'parent']
on ['collector'] | ['el', 'event', 'fn']
on ['collector'] | ['el', 'event', 'fn']
add ['txt'] | ['manager', 'name', 'password']
add ['self2'] | ['manager', 'name', 'password']
on ['rsp'] | ['el', 'event', 'fn']
add ['box'] | ['manager', 'name', 'password']
renderMap ['geo_data', 'year'] | ['selector', 'geo_data', 'wb_data', 'year']
renderMap ['geo_data'] | ['selector', 'geo_data', 'wb_data', 'year']
